# CIS545: Final Project - CORD-19 Dataset
## Justin Choi
## TA: Hoyt Gong

Hi there! For this project, I opted to use the COVID-19 research challenge dataset, which they named "CORD-19" (I promise, the title wasn't a typo haha), and below you'll see the respective EDA and modeling I've done for the dataset! Hope you enjoy ~


In [30]:
import pandas as pd 
import numpy as np 
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import glob, os
import json
import nltk


# The Data 
Thanks to all the ~ useful ~ skills we've picked up over the course of this semester, we'll start with everyone's favorite tedious, time-consuming task - data cleaning! woo hoooooOOOOooOo who doesn't lüv missing values and weird formatting. 

Most of the work in this section was basically just finding good ways to extract the text from the json format to put into the dataframe, and then proceeded to clean it using basic `pandas` and the `langdetect` package in order to filter out any articles not in English in order to standardize the data and simplify visualization/processing/modeling later on! 

In [31]:
metadata_df = pd.read_csv('./CORD-19-research-challenge/metadata.csv')

# Import all the json files
cord_19_folder = './CORD-19-research-challenge/'
list_of_files = []; # only going to take those from pdf_json! not pmc_json
for root, dirs, files in os.walk(cord_19_folder):
    for name in files:
        if (name.endswith('.json')):
            full_path = os.path.join(root, name)
            list_of_files.append(full_path)
sorted(list_of_files)
print('done')

# ALTERNATE

# all_json = glob.glob(f'{cord_19_folder}/**/*.json', recursive=True)
# len(all_json)


done


In [32]:
class JsonReader:
    def __init__(self, file_path):
        with open(file_path) as file: 
            content = json.load(file)
            # start to insert body text 
            self.paper_id = content['paper_id']
            self.body_text = [] 
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.body_text[:500]}...'

random_json = list_of_files[47404]
sample_article = JsonReader(random_json)
print(sample_article)

PMC1403753: Migration, transmigration, [1] return migration, and remigration constitute defining elements of the current and future world order. More than 700 million people (including visitors on business or personal/family trips) traverse nation-state borders annually [2,3] and one million per week move between the global South and the global North [4]. The enormity of contemporary transnational mobility is illustrated by the case of Australia. In the past half century, Australia's "resident population ha...


In [33]:
input = {'paper_id': [], 'doi':[],  'title': [], 'abstract': [], 'body_text': [], 'authors': [], 'journal': []}

for index, entry in enumerate(list_of_files):
    if index % (len(list_of_files) // 10) ==    0:
        print(f'Processing {index} of {len(list_of_files)}')
    try: 
        article = JsonReader(entry)
    except Exception as e: 
        continue #means that we don't have a valid file format  
    
    metadata = metadata_df.loc[metadata_df['sha'] == article.paper_id]
    if len(metadata) == 0:
        continue # no such metadata for paper in our csv, skip

    input['body_text'].append(article.body_text)
    input['paper_id'].append(article.paper_id)

    # add in metadata 
    title = metadata['title'].values[0] 
    doi = metadata['doi'].values[0] 
    abstract = metadata['abstract'].values[0] 
    authors = metadata['authors'].values[0] 
    journal = metadata['journal'].values[0] 

    input['title'].append(title)
    input['doi'].append(doi)
    input['abstract'].append(abstract)
    input['authors'].append(authors)
    input['journal'].append(journal)



Processing 0 of 59311
Processing 5931 of 59311
Processing 11862 of 59311
Processing 17793 of 59311
Processing 23724 of 59311
Processing 29655 of 59311
Processing 35586 of 59311
Processing 41517 of 59311
Processing 47448 of 59311
Processing 53379 of 59311
Processing 59310 of 59311


In [85]:
covid_df = pd.DataFrame(input, columns=['paper_id', 'doi', 'title', 'abstract', 'body_text', 'authors', 'journal'])
print('finished creating dataframe from input dictionary')
rows, cols = covid_df.shape
print(f'number of rows: {rows}')
covid_df.info()

finished creating dataframe from input dictionary
number of rows: 36009
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36009 entries, 0 to 36008
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   paper_id   36009 non-null  object
 1   doi        35672 non-null  object
 2   title      35973 non-null  object
 3   abstract   31675 non-null  object
 4   body_text  36009 non-null  object
 5   authors    35413 non-null  object
 6   journal    34277 non-null  object
dtypes: object(7)
memory usage: 1.9+ MB


In [86]:
covid_df.dropna(inplace=True)
print('finished dropping articles with null abstracts/body text/titles')
rows, cols = covid_df.shape
print(f'number of rows: {rows}')

finished dropping articles with null abstracts/body text/titles
number of rows: 29600


In [87]:
covid_df['abstract_word_count'] = covid_df['abstract'].apply(lambda x : len(x.strip().split()))
covid_df['body_word_count'] = covid_df['body_text'].apply(lambda x : len(x.strip().split()))
covid_df['body_unique_count'] = covid_df['body_text'].apply(lambda x : len(set(x.strip().split())))


In [88]:
# visualiation check to see if data is finished being cleaned 
covid_df.head()


,paper_id,doi,title,abstract,body_text,authors,journal,abstract_word_count,body_word_count,body_unique_count
0,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,10.1080/14787210.2017.1271712,Update on therapeutic options for Middle East ...,Introduction: The Middle East Respiratory Synd...,The Middle East respiratory syndrome coronavir...,"Al-Tawfiq, Jaffar A.; Memish, Ziad A.",Expert Rev Anti Infect Ther,174,2748,996
1,306ef95a3a91e13a93bcc37fb2c509b67c0b5640,10.1093/cid/ciaa256,A Novel Approach for a Novel Pathogen: using a...,Thousands of people in the United States have ...,"The 2019 novel coronavirus (SARS-CoV-2), ident...","Bryson-Cahn, Chloe; Duchin, Jeffrey; Makarewic...",Clin Infect Dis,50,944,486
2,ab680d5dbc4f51252da3473109a7885dd6b5eb6f,10.1016/b978-0-12-800049-6.00293-6,Evolutionary Medicine IV. Evolution and Emerge...,Abstract This article discusses how evolutiona...,The evolutionary history of humans is characte...,"Scarpino, S.V.",Encyclopedia of Evolutionary Biology,72,2884,1091
3,6599ebbef3d868afac9daa4f80fa075675cf03bc,10.1016/j.enpol.2008.08.029,International aviation emissions to 2025: Can ...,Abstract International aviation is growing rap...,"Sixty years ago, civil aviation was an infant ...","Macintosh, Andrew; Wallace, Lailey",Energy Policy,137,5838,1587
5,44290ff75bad8ffaf5d3028420739ce7b08dc2a9,10.1093/jac/dkp502,Inhibition of enterovirus 71 replication and t...,OBJECTIVES: Enterovirus 71 (EV71) causes serio...,Enteroviruses are members of the family Picorn...,"Hung, Hui-Chen; Chen, Tzu-Chun; Fang, Ming-Yu;...",J Antimicrob Chemother,260,3121,1064


In [89]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29600 entries, 0 to 36008
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   paper_id             29600 non-null  object
 1   doi                  29600 non-null  object
 2   title                29600 non-null  object
 3   abstract             29600 non-null  object
 4   body_text            29600 non-null  object
 5   authors              29600 non-null  object
 6   journal              29600 non-null  object
 7   abstract_word_count  29600 non-null  int64 
 8   body_word_count      29600 non-null  int64 
 9   body_unique_count    29600 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.5+ MB


In [91]:
#check to see if there are duplicates
covid_df['abstract'].describe()

count       29600
unique      29543
top       Unknown
freq           21
Name: abstract, dtype: object

In [92]:
covid_df.drop_duplicates(subset=['abstract', 'body_text'], inplace=True)

In [93]:
covid_df['abstract'].describe()

count       29597
unique      29543
top       Unknown
freq           21
Name: abstract, dtype: object

In [94]:
covid_df['body_text'].describe()

count                                                 29597
unique                                                29591
top       The importance of foodborne transmission of vi...
freq                                                      2
Name: body_text, dtype: object

In [131]:
covid_df.describe()

,abstract_word_count,body_word_count,body_unique_count
count,29597.000000,29597.000000,29597.000000
mean,194.711356,4560.325878,1425.199007
std,151.302225,3528.817632,748.579044
min,1.000000,2.000000,2.000000
25%,140.000000,2705.000000,989.000000
50%,191.000000,3847.000000,1288.000000
75%,243.000000,5533.000000,1695.000000
max,18000.000000,171948.000000,25156.000000


In [168]:
from langdetect import detect
from tqdm import tqdm

languages = [] # make list that you can port directly into covid_df as column

for i in tqdm(range(len(covid_df))):
    row_text = covid_df.iloc[i]['body_text'].split(" ") 
    lang = 'en' # set default lang to be english 

    # try to just use the intro 25 words to detect language
    try:
        if (len(row_text)) > 25: 
            lang = detect(" ".join(row_text[:25]))
        elif(len(row_text)) > 0:
            lang = detect(" ".join(row_text))
    except Exception as e: # if body doesn't work, let's try abstract
        try: 
            lang = detect(covid_df.iloc(i)['abstract'])
        except Exception as e:
            lang = 'dunno'
            continue
    finally:
        languages.append(lang)
        

100%|██████████| 29597/29597 [02:19<00:00, 212.16it/s]


In [201]:
lang_dict = {}
for language in set(languages):
    lang_dict[language] = languages.count(language)

print(lang_dict)

{'zh-cn': 1, 'de': 69, 'dunno': 1, 'fr': 248, 'es': 165, 'ca': 5, 'tl': 2, 'da': 1, 'sv': 1, 'pt': 17, 'en': 29029, 'pl': 2, 'ro': 2, 'it': 16, 'id': 1, 'nl': 37}


In [202]:
lang_array = np.asarray(languages)
covid_df['language'] = lang_array

ValueError: Length of values does not match length of index

In [203]:
covid_df.head()

,paper_id,doi,title,abstract,body_text,authors,journal,abstract_word_count,body_word_count,body_unique_count,language
0,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,10.1080/14787210.2017.1271712,Update on therapeutic options for Middle East ...,Introduction: The Middle East Respiratory Synd...,The Middle East respiratory syndrome coronavir...,"Al-Tawfiq, Jaffar A.; Memish, Ziad A.",Expert Rev Anti Infect Ther,174,2748,996,en
1,306ef95a3a91e13a93bcc37fb2c509b67c0b5640,10.1093/cid/ciaa256,A Novel Approach for a Novel Pathogen: using a...,Thousands of people in the United States have ...,"The 2019 novel coronavirus (SARS-CoV-2), ident...","Bryson-Cahn, Chloe; Duchin, Jeffrey; Makarewic...",Clin Infect Dis,50,944,486,en
2,ab680d5dbc4f51252da3473109a7885dd6b5eb6f,10.1016/b978-0-12-800049-6.00293-6,Evolutionary Medicine IV. Evolution and Emerge...,Abstract This article discusses how evolutiona...,The evolutionary history of humans is characte...,"Scarpino, S.V.",Encyclopedia of Evolutionary Biology,72,2884,1091,en
3,6599ebbef3d868afac9daa4f80fa075675cf03bc,10.1016/j.enpol.2008.08.029,International aviation emissions to 2025: Can ...,Abstract International aviation is growing rap...,"Sixty years ago, civil aviation was an infant ...","Macintosh, Andrew; Wallace, Lailey",Energy Policy,137,5838,1587,en
5,44290ff75bad8ffaf5d3028420739ce7b08dc2a9,10.1093/jac/dkp502,Inhibition of enterovirus 71 replication and t...,OBJECTIVES: Enterovirus 71 (EV71) causes serio...,Enteroviruses are members of the family Picorn...,"Hung, Hui-Chen; Chen, Tzu-Chun; Fang, Ming-Yu;...",J Antimicrob Chemother,260,3121,1064,en


In [233]:
covid_df = covid_df[covid_df['language'] == 'en'] # i.e. only select articles written in english, as it'll help parsing/NLP 

,paper_id,doi,title,abstract,body_text,authors,journal,abstract_word_count,body_word_count,body_unique_count,language
0,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,10.1080/14787210.2017.1271712,Update on therapeutic options for Middle East ...,Introduction: The Middle East Respiratory Synd...,The Middle East respiratory syndrome coronavir...,"Al-Tawfiq, Jaffar A.; Memish, Ziad A.",Expert Rev Anti Infect Ther,174,2748,996,en
1,306ef95a3a91e13a93bcc37fb2c509b67c0b5640,10.1093/cid/ciaa256,A Novel Approach for a Novel Pathogen: using a...,Thousands of people in the United States have ...,"The 2019 novel coronavirus (SARS-CoV-2), ident...","Bryson-Cahn, Chloe; Duchin, Jeffrey; Makarewic...",Clin Infect Dis,50,944,486,en
2,ab680d5dbc4f51252da3473109a7885dd6b5eb6f,10.1016/b978-0-12-800049-6.00293-6,Evolutionary Medicine IV. Evolution and Emerge...,Abstract This article discusses how evolutiona...,The evolutionary history of humans is characte...,"Scarpino, S.V.",Encyclopedia of Evolutionary Biology,72,2884,1091,en
3,6599ebbef3d868afac9daa4f80fa075675cf03bc,10.1016/j.enpol.2008.08.029,International aviation emissions to 2025: Can ...,Abstract International aviation is growing rap...,"Sixty years ago, civil aviation was an infant ...","Macintosh, Andrew; Wallace, Lailey",Energy Policy,137,5838,1587,en
5,44290ff75bad8ffaf5d3028420739ce7b08dc2a9,10.1093/jac/dkp502,Inhibition of enterovirus 71 replication and t...,OBJECTIVES: Enterovirus 71 (EV71) causes serio...,Enteroviruses are members of the family Picorn...,"Hung, Hui-Chen; Chen, Tzu-Chun; Fang, Ming-Yu;...",J Antimicrob Chemother,260,3121,1064,en


In [236]:
covid_df['language'].describe()

count     29029
unique        1
top          en
freq      29029
Name: language, dtype: object

# NLP + feature extraction
Now that we've successfully cleaned out the data and gotten all the text in a consistent fashion, now we're going to create a bag-of-words model and vectorize each of the documents! This'll then allow us to do some better visualization and run some cool ~__machine learning__~ like PCA and t-SNE to both reduce dimensionality and visualize this better! 

In [ ]:
# NLP analysis using Spacy
import spacy 
from space.lang.en.stop_words import STOP_WORDS

stop_words = STOP_WORDS
print(stop_words)